In [1]:
import json, sys, re, platform, os
from winpython import utils
from collections import OrderedDict
from pip._vendor.packaging.markers import Marker


def normalize(this):
    """apply https://peps.python.org/pep-0503/#normalized-names"""
    return re.sub(r"[-_.]+", "-", this).lower()


pip_inspect = utils.exec_run_cmd(["pip", "inspect"])
pip_json = json.loads(pip_inspect)


# create a dict with
#  key = normalised package name
#     string_elements = 'name', 'version', 'summary'
#     requires =  list of dict with 1 level need downward
#             req_key = package_key requires
#             req_extra = extra branch needed of the package_key ('all' or '')
#             req_version = version needed
#             req_marker = marker of the requirement (if any)
distro = {}
replacements = str.maketrans({" ": "", "[": "", "]": "", "'": "", '"': ""})
environment = {
    "implementation_name": sys.implementation.name,
    "implementation_version": "{0.major}.{0.minor}.{0.micro}".format(
        sys.implementation.version
    ),
    "os_name": os.name,
    "platform_machine": platform.machine(),
    "platform_release": platform.release(),
    "platform_system": platform.system(),
    "platform_version": platform.version(),
    "python_full_version": platform.python_version(),
    "platform_python_implementation": platform.python_implementation(),
    "python_version": ".".join(platform.python_version_tuple()[:2]),
    "sys_platform": sys.platform,
}


for p in pip_json["installed"]:
    meta = p["metadata"]
    name = meta["name"]
    key = normalize(name)
    requires = []
    if "requires_dist" in meta:
        for i in meta["requires_dist"]:
            det = (i + ";").split(";")
            req_nameextra = normalize((det[0] + " ").split(" ")[0])
            req_key = normalize((req_nameextra + "[").split("[")[0])
            req_key_extra = req_nameextra[len(req_key) + 1 :].split("]")[0]
            req_version = det[0][len(req_nameextra) :].translate(replacements)
            req_marker = det[1]

            req_add = {
                "req_key": req_key,
                "req_version": req_version,
                "req_extra": req_key_extra,
            }
            # add the marker of the requirement, if not nothing:
            if not req_marker == "":
                req_add["req_marker"] = req_marker
            requires += [req_add]
    distro[key] = {
        "name": name,
        "version": meta["version"],
        "summary": meta["summary"] if "summary" in meta else "",
        "requires_dist": requires,
        "wanted_per": [],
    }
# On a second pass, complement distro in reverse mode with 'wanted-per':
# - get all downward links in 'requires_dist' of each package
# - feed the required packages 'wanted_per' as a reverse dict of dict
#        contains =
#             req_key = upstream package_key
#             req_version = downstream package version wanted
#             req_marker = marker of the downstream package requirement (if any)

for p in distro:
    for r in distro[p]["requires_dist"]:
        if r["req_key"] in distro:
            want_add = {
                "req_key": p,
                "req_version": r["req_version"],
                "req_extra": r["req_extra"],
            }  # req_key_extra
            if "req_marker" in r:
                want_add["req_marker"] = r["req_marker"]  # req_key_extra
            distro[r["req_key"]]["wanted_per"] += [want_add]


def pipdownraw(pp, extra="", version_req="", depth=20, path=[]):
    """build downward requirements for the package on given extra and depth"""
    envi = {"extra": extra, **environment}
    p = normalize(pp)
    ret_all = []
    if p in path:
        print("cycle!", "->".join(path + [p]))
    elif p in distro and len(path) <= depth:
        if extra == "":
            ret = [f'{p}=={distro[p]["version"]} {version_req}']
        else:
            ret = [f'{p}[{extra}]=={distro[p]["version"]} {version_req}']
        for r in distro[p]["requires_dist"]:
            if r["req_key"] in distro:
                if "req_marker" not in r or Marker(r["req_marker"]).evaluate(
                    environment=envi
                ):
                    ret += pipdownraw(
                        r["req_key"],
                        r["req_extra"],
                        r["req_version"],
                        depth,
                        path + [p],
                    )
        ret_all += [ret]
    return ret_all


def pipupraw(pp, extra="", version_req="", depth=20, path=[]):
    """build upward needs for the package on given extra and depth"""
    envi = {"extra": extra, **environment}
    p = normalize(pp)
    ret_all = []
    if p in path:
        print("cycle!", "->".join(path + [p]))
    if p in distro and len(path) <= depth:
        if extra == "":
            ret_all = [f'{p}=={distro[p]["version"]} {version_req}']
        else:
            ret_all = [f'{p}[{extra}]=={distro[p]["version"]} {version_req}']
        ret = []
        for r in distro[p]["wanted_per"]:
            # print(distro[p]['wanted_per'][r]['req,_version'])
            if r["req_key"] in distro and r["req_key"] not in path:
                # print(r)
                if "req_marker" not in r or Marker(r["req_marker"]).evaluate(
                    environment=envi
                ):
                    ret += pipupraw(
                        r["req_key"],
                        "",
                        f"[requires: {p}"
                        + ("[" + r["req_extra"] + "]" if r["req_extra"] != "" else "")
                        + f'{r["req_version"]}]',
                        depth,
                        path + [p],
                    )
        if not ret == []:
            ret_all += [ret]
    return ret_all


def pipdown(pp="", extra="", depth=99, indent=5, version_req=""):
    """print the downward requirements for the package or all packages"""
    if not pp == "":
        rawtext = json.dumps(pipdownraw(pp, extra, version_req, depth), indent=indent)
        lines = [l for l in rawtext.split("\n") if len(l.strip()) > 2]
        print("\n".join(lines).replace('"', ""))
    else:
        for one_pp in sorted(distro):
            pipdown(one_pp, extra, depth, indent, version_req)


def pipup(pp, extra="", depth=99, indent=5, version_req=""):
    """print the upward needs for the package"""
    rawtext = json.dumps(pipupraw(pp, extra, version_req, depth), indent=indent)
    lines = [l for l in rawtext.split("\n") if len(l.strip()) > 2]
    print("\n".join(lines).replace('"', ""))


In [2]:
pipdown('pandas')

          pandas==1.4.3 ,
               python-dateutil==2.8.2 (>=2.8.1),
                    six==1.16.0 (>=1.5)
               pytz==2022.1 (>=2020.1)
               numpy==1.22.4 (>=1.21.0)


In [3]:
pipdown('pandas', 'test')

          pandas[test]==1.4.3 ,
               python-dateutil==2.8.2 (>=2.8.1),
                    six==1.16.0 (>=1.5)
               pytz==2022.1 (>=2020.1)
               numpy==1.22.4 (>=1.21.0)
               hypothesis==6.46.9 (>=5.5.3),
                    attrs==22.1.0 (>=19.2.0)
                    sortedcontainers==2.4.0 (<3.0.0,>=2.1.0)
               pytest==7.1.1 (>=6.0),
                    attrs==22.1.0 (>=19.2.0)
                    iniconfig==1.1.1 
                    packaging==21.3 ,
                         pyparsing==2.4.7 (!=3.0.5,>=2.0.2)
                    pluggy==1.0.0 (<2.0,>=0.12)
                    py==1.11.0 (>=1.8.2)
                    tomli==2.0.1 (>=1.0.0)
                    atomicwrites==1.4.0 (>=1.0)
                    colorama==0.4.4 


In [4]:
pipup('pytest') # collected data for packag

     pytest==7.1.1 ,
          nbval==0.9.6 [requires: pytest(>=2.8)]


In [5]:
pipup('pytest', 'test', 1)

     pytest[test]==7.1.1 ,
          wheel==0.37.1 [requires: pytest(>=3.0.0)],
          validators==0.18.2 [requires: pytest(>=2.2.3)],
          tzlocal==4.2 [requires: pytest(>=4.3)],
          typer==0.4.2 [requires: pytest>=4.4.0,<5.4.0],
          traittypes==0.2.1 [requires: pytest],
          traitlets==5.3.0 [requires: pytest],
          tinycss2==1.1.1 [requires: pytest],
          three-merge==0.1.1 [requires: pytest],
          textdistance==4.2.2 [requires: pytest],
          sqlite-utils==3.26 [requires: pytest],
          sqlite-fts4==1.0.3 [requires: pytest],
          spyder==5.4.0.dev0 [requires: pytest(<7.0)],
          spyder-kernels==2.3.2 [requires: pytest],
          sphinxcontrib-serializinghtml==1.1.5 [requires: pytest],
          sphinxcontrib-qthelp==1.0.3 [requires: pytest],
          sphinxcontrib-jsmath==1.0.1 [requires: pytest],
          sphinxcontrib-htmlhelp==2.0.0 [requires: pytest],
          sphinxcontrib-devhelp==1.0.2 [requires: pytest],
        

In [6]:
# to inquire on what 'pip inspect' gives for list of pakage list_of_package
list_of_package = ['ipython',] # aiohttp', 'pandas', 'Pandas', 'dask', 'Dask', 'Spyder', 'spyder']:

for p in pip_json['installed']:
    meta = p['metadata']
    if  meta['name'] in list_of_package:
         # print(json.dumps(meta).split(r'\n'))
         print('\n'.join(meta['description'].split(r'\n')))


IPython provides a rich toolkit to help you make the most out of using Python
interactively.  Its main components are:

* A powerful interactive Python shell
* A `Jupyter <https://jupyter.org/>`_ kernel to work with Python code in Jupyter
  notebooks and other interactive frontends.

The enhanced interactive Python shells have the following main features:

* Comprehensive object introspection.

* Input history, persistent across sessions.

* Caching of output results during a session with automatically generated
  references.

* Extensible tab completion, with support by default for completion of python
  variables and keywords, filenames and function keywords.

* Extensible system of 'magic' commands for controlling the environment and
  performing many tasks related either to IPython or the operating system.

* A rich configuration system with easy switching between different setups
  (simpler than changing $PYTHONSTARTUP environment variables every time).

* Session logging and rel